<a href="https://colab.research.google.com/github/mirrtl/deepl/blob/main/TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Оценка модели mini-jenny-30H в моей работе будет осуществляться как стандартными методами, так и методами направленными на оценку специфик голоса jenny.

**Для работы кода надо скачать прикрепленный файл с тестовым аудио или же использовать свое(затем изменить путь на аудио). Также прикреплен файл со сгенерированным аудио, на котором проводилась оценка.**

**План оценки модели**
Отбор аудио состоял из нахождения отрывка речи, в моем случае это был отрывок с чтением стиха, с голосом и интонацией максимально похожими на голос Дженни. Эти факторы играют большую роль в двух метриках, которые будут рассмотрены здесь

**1 метрика - WER**

WER (Word Error Rate) — это метрика, которая используется для оценки качества распознанной речи или текстового результата TTS (Text-to-Speech) моделей. Она показывает, насколько сильно сгенерированный текст отличается от эталонного (референсного) текста.

**2 метрика - STOI**

Метрика STOI(Short-Time Objective Intelligibility) Измеряет корреляцию между огибающими двух сигналов. В нашем случае сигналы это записи голосов(идеальный и сгенерированный).

**3 метрика**

Эта метрика показывает отношения выбросов частот в спектре голосов к общему числу частей спектра. То есть показывает процентное соотношение выбросов.
Реализуется так, что выбросами частоты считаются, если отклоняются от средней частоты более чем на 3 стандартных отклонения и если амплитуда этих колебания превышает определенный порог.

**Причина выбора метрик и описание**

Первая и третья метрика, позволяют хорошо оценить сгенерированное аудио без отобранного аудио, они позволяют выявить ошибки на уровне самой генерации(выбросы, шумы и тд.) и узнать прошла ли генерация самого текста без ошибок.

2 метрика позволяет оценить как насколько похожи сгенерированное аудио с тем аудио, которое мы отобрали с целью сгенерировать похожее.
Она покажет степень похожести двух аудио

**Промт и описание для генерации**

 *prompt = '''The movement of your hands is the long, golden running of light from a rising sun;
It is the hopping of birds upon a garden-path;
As the perfume of jonquils, you come forth in the morning.'''*

*description = "Jenny speaks at a slow pace with an animated delivery in a confined environment with clear audio, audio must be 15 seconds long"*

Описание было выбрано таким, поскольку выбранное аудио состояло из чтения стиха в медленном темпе, без шумов и в ограниченном диапазоне частот. Так же указался желательное время длины аудио


In [ ]:
pip install git+https://github.com/huggingface/parler-tts.git

In [ ]:
!pip install jiwer

In [ ]:
!pip install SpeechRecognition

In [ ]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-mini-jenny-30H").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-mini-jenny-30H", use_fast=True)
# Текст, на основе которого будет проводиться генерация
prompt = '''The movement of your hands is the long, golden running of light from a rising sun;
It is the hopping of birds upon a garden-path;
As the perfume of jonquils, you come forth in the morning.'''
# Описание критериев для генерации
description = "Jenny speaks at a slow pace with an animated delivery in a confined environment with clear audio, audio must be 15 seconds long"

input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
audio_arr = generation.cpu().numpy().squeeze()
sf.write("parler_tts_out.wav", audio_arr, model.config.sampling_rate)

#**1 метрика - WER**


In [18]:
import jiwer
from speech_recognition import Recognizer, AudioFile, UnknownValueError, RequestError
from jiwer import Compose

# Создание кастомного трансформера для предобработки текстов
custom_transform = Compose([
    jiwer.RemovePunctuation(),  # Удалить пунктуацию
    jiwer.ToLowerCase(),        # Преобразовать в нижний регистр
    jiwer.RemoveMultipleSpaces(),# Удалить лишние пробелы
    lambda s: s.replace("\n", " ") # Удалить символы перевода на новую строку
])

recognizer = Recognizer()

# Исходный текст
ground_truth = '''The movement of your hands is the long, golden running of light from a rising sun;
It is the hopping of birds upon a garden-path;
As the perfume of jonquils, you come forth in the morning.'''
recognized_texts = []
# Распознавание аудио
with AudioFile("parler_tts_out.wav") as source:
    audio = recognizer.record(source)
recognized_text = recognizer.recognize_google(audio, language="en-US")
recognized_texts.append(recognized_text)


# Применение кастомного трансформера
ground_truth_transformed = custom_transform(ground_truth)
recognized_texts_transformed = [custom_transform(rt) for rt in recognized_texts]

# Вычисление WER
wer = jiwer.wer(ground_truth_transformed, recognized_texts_transformed[0])
print(f"Word Error Rate (WER): {wer:.2f}")

# Вывод распознанных текстов
print(f"Ground Truth: {ground_truth_transformed}")
print(f"Recognized  : {recognized_texts_transformed[0]}")

Word Error Rate (WER): 0.08
Ground Truth: the movement of your hands is the long golden running of light from a rising sun it is the hopping of birds upon a gardenpath as the perfume of jonquils you come forth in the morning
Recognized  : the movements of your hands is the long golden running of light from a rising sun it is the hopping of birds upon a garden path as the perfume of jonquils you come forth in the morning


#**2 метрика - STOI**

In [20]:
import numpy as np
import librosa
import pystoi

# Загрузить тестовое и сгенерированное аудио с разными частотами дискретизации
reference, rate_ref = librosa.load('spc249_inexcelsis_ams_64kb (1) (mp3cut.net).wav', sr=16000)
degraded, rate_deg = librosa.load('parler_tts_out.wav', sr=16000)

min_len = min(reference.shape[0], degraded.shape[0])
reference = reference[:min_len]
degraded = degraded[:min_len]

# STOI оценка
stoi_score = pystoi.stoi(reference, degraded, rate_ref, extended=False)
print(f"STOI оценка: {stoi_score:.2f}")


STOI оценка: 0.22


# **3 метрика**


Во время анализирования задания, появилась идея реализовать метрику, которая находила бы выбросы в спектре голоса, те значения частоты которые не должны соответствовать голосу Дженни. Такая метрика в этом случае (скорее всего) должна показывать хорошие результаты, поскольку сгенерированная запись голоса находится примерно в одной тональности, и каких-то высоких и низких частот не должно присутствовать. Значения частот спектра считаются выбросом, если амплитуда колебания превышают определенный уровень amplitude_threshold

In [21]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

# Загрузка аудиофайла
y, sr = librosa.load("parler_tts_out.wav", sr=None)

# Вычисление спектрограммы
S = librosa.stft(y)
S_db = librosa.amplitude_to_db(np.abs(S), ref=np.max)

# Вычисление среднего спектра и стандартного отклонения
mean_spectrum = np.mean(S_db, axis=1)  # Среднее по времени
std_spectrum = np.std(S_db, axis=1)    # Стандартное отклонение по времени

# Порог для выбросов
threshold = mean_spectrum + 3 * std_spectrum

# Минимальный порог амплитуды (например, -60 dB)
amplitude_threshold = -60

# Определение выбросов
anomalies = (S_db > threshold[:, np.newaxis]) & (S_db > amplitude_threshold)

# Количество выбросов
num_anomalies = np.sum(anomalies)

# Общее количество точек
total_points = S_db.size

# Доля выбросов
fraction_anomalies = num_anomalies / total_points

print(f"Доля выбросов: {fraction_anomalies:.4f}")

Доля выбросов: 0.0089


#**Отчет**

Сделаем выводы по полученным метрикам.

**1 метрика - WER**

wer = 0.08 - доля неверно написанных слов. Означает что 8% слов написано неверно. В нашем случае модель распознования речи посчитала что слово groundpath должно писать раздельно как ground path. За ошибку такую неточность можно не считать

**2 метрика - STOI**

STOI = 0.22. В идеальном случае метрика равна 1. По такому значению можно сказать, что какая-то корреляцию все же есть, но он очень мала. Это происходит из-за того, что модель не смогла подобрать очень похожую интонацию, поскольку корреляция определяется по времени. Благодаря такой метрике можно говорить о правильности подобранной итонации.

**3 метрика**
метрика = 0.0089. Идеальным значением является 0. В нашем случае метрика очень близка к ней, что говорит о малом колличестве шумов и статистически аномальных значения спектра.

